In [45]:
# Import necessary libraries
import ephem
import math


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    pass_data = {}
    i = 0

    try:
        if num_passes is not None and duration is None:
            # if only num_passes specified
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                """next pass command yields array with [0]=rise time,
                [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                [4]=set time, [5]=set azimuth"""
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data[i] = [rise_time, set_time, pass_duration,
                                (rise_az*deg_per_rad), (set_az*deg_per_rad)]

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
            return pass_data

        if num_passes is None and duration is not None:
            # if only duration specified
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                """next pass command yields array with [0]=rise time,
                [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                [4]=set time, [5]=set azimuth"""
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data[i] = [rise_time, set_time, pass_duration,
                                (rise_az*deg_per_rad), (set_az*deg_per_rad)]

                i = i + 1
                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
            return pass_data

        if num_passes is not None and duration is not None:
            # if both are specified, use num_passes
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                """next pass command yields array with [0]=rise time,
                [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                [4]=set time, [5]=set azimuth"""
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data[i] = [rise_time, set_time, pass_duration,
                                (rise_az*deg_per_rad), (set_az*deg_per_rad)]

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
            return pass_data

        if num_passes is None and duration is None:
            # if neither are specified, get passes for the next 24 hours
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                """next pass command yields array with [0]=rise time,
                [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                [4]=set time, [5]=set azimuth"""
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data[i] = [rise_time, set_time, pass_duration,
                                (rise_az*deg_per_rad), (set_az*deg_per_rad)]

                i = i + 1
                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
            return pass_data
    except 

SyntaxError: unexpected EOF while parsing (<ipython-input-45-d27703ab838c>, line 130)

In [39]:
import requests

#def file_len(fname):
#    """Return lengths of file"""
#    with open(fname) as f:
#        for i, l in enumerate(f):
#            pass
#    return i + 1

In [40]:
# itertools has lots of things that deal with looping
# "generators" and "iterators" are really nice concepts that
#can be used in place of most usage of using indexes in a loop
#because usually we aren't actually interested in counting the lines
from itertools import islice
data = []
with open('amateur.txt') as f:
    #we don't know we've reached the end of the file
    #until we actually reached it
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        # this is called a "list comprehension"
        # it is pretty much exactly the same as
        #    tle = []
        #    for line in tripleline:
        #      tle.append(x.rstrip())
        #
        tle = [x.rstrip() for x in tripleline]
        
        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break

# now, data is a list of 3-element lists of strings
data

[['OSCAR 7 (AO-7)',
  '1 07530U 74089B   17157.85566058 -.00000032  00000-0  74274-4 0  9991',
  '2 07530 101.6293 126.7439 0012026 344.7437  44.8584 12.53627301947333'],
 ['UOSAT 2 (UO-11)',
  '1 14781U 84021B   17157.57376735  .00000098  00000-0  18371-4 0  9991',
  '2 14781  97.6981 203.6799 0007351 227.5873 132.4720 14.82898575767779'],
 ['LUSAT (LO-19)',
  '1 20442U 90005G   17157.80919745 -.00000011  00000-0  11872-4 0  9996',
  '2 20442  98.5753  94.4288 0011798 172.7670 187.3686 14.32861011430072'],
 ['EYESAT-1 (AO-27)',
  '1 22825U 93061C   17157.83712816 -.00000010  00000-0  13989-4 0  9991',
  '2 22825  98.8112 123.2914 0008598   9.6571 350.4775 14.29997613235653'],
 ['ITAMSAT (IO-26)',
  '1 22826U 93061D   17157.87055661 -.00000009  00000-0  13959-4 0  9999',
  '2 22826  98.8055 123.4085 0008931 359.6761   0.4413 14.30326285236106'],
 ['RADIO ROSTO (RS-15)',
  '1 23439U 94085A   17157.65856149 -.00000029  00000-0  38782-3 0  9995',
  '2 23439  64.8155 236.1967 0167137 247.5

In [44]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 12:00:00'
num_passes = 20
duration = None
sat_passes = []

for tle in data:
    sat_passes.append(get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration))

ValueError: that satellite seems to stay always below your horizon